### Full Range

策略： 在full range内提供流动性

In [330]:
import sys
sys.path.append("..") 
import pandas as pd
from datetime import datetime
from importlib import reload
from poolData import swapData
from utils import utils
reload(swapData)
reload(utils)

query = swapData.SwapDataQuery()
utils = utils.utils()

In [276]:
# position info
# 选定池子： token0: usdc(decimal: 6), token1: weth(decimal: 18)
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
query.query_positions(pool_id=pool_id, block_gte=16859000, limit=10, orderBy="owner")
decimal0 = 6
decimal1 = 18
# 初始头寸的美元价值
initial_position = 100000

# 选定时间
begin = datetime(2023, 3, 19)
end = datetime.now()

pool_id: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 - type: <class 'str'>


In [278]:
# 获取起始时间和结束时间的价格
# 假设本金是1000刀，token0是usdc，数量是500
token0_amount = initial_position / 2
# 计算token1数量的时候需要知道价格
# 获取价格
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640")
liquidity_data
price_begin = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][0]))
price_end = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][liquidity_data.shape[0]-1]))
price_begin_real = utils.price2RealPrice(price_begin, decimal0=decimal0, decimal1=decimal1)
price_end_real = utils.price2RealPrice(price_end, decimal0=decimal0, decimal1=decimal1)
# 拿到价格之后才可以计算初始头寸
token1_amount = token0_amount * price_begin_real

this is the cursor: 1682499600


### 计算该策略的手续费

首先需要计算出头寸的liquidity是多少。

full range策略相当于v2中的头寸，在全价格范围内提供流动性，在v3中就是minTick和maxTick(0,infinity)之间提供流动性。

我在测试网上提供了一个full range的[头寸](https://goerli.etherscan.io/tx/0x199a0a53afbaa7c6b542dc2bdc2766639e9250055032238ebcf204d7375c8aa5), 得到：

tickLower： -887270
tickUpper： 887270


In [186]:
# 计算价格范围
tickLower = -887270
tickUpper = 887270
price_low = utils.tickIndex2Price(tickLower)
price_upper = utils.tickIndex2Price(tickUpper)
print("this is the lower price: ", price_low)
print("this is the upper price: ", price_upper)

this is the lower price:  2.939544628365392e-39
this is the upper price:  3.4018874568203963e+38


In [293]:
# 计算liquidity
l = utils.calculate_fullRange_liquidity(token0_amount, token1_amount, decimal0, decimal1)
print("l_fullRange:", l)

l_fullRange: 1181187906714923.5


In [236]:
# 获取选定时间期限内的所有swaps
swaps = query.query_swaps(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)

this is the cursor: 0x0118c203ac60097b3c2602a8b90bb50d6b3a93516826f9ee1849cc72b35e07e4#4732559
this is the cursor: 0x02392fc760d8eb9a82ee1200195009b22026ee7bc68346dad2a09b6a0f708108#4634833
this is the cursor: 0x034bad187bdf7884cf643b496fbf7a2a6bf0295d5e8c0f1cc77f6e92d4963473#4789891
this is the cursor: 0x047e5a6d0fc3bf2f1e56de331f9f88e2f8cca6f119a9febfb5fc475ce596d01f#4788463
this is the cursor: 0x05a0543bb429daa9c4bce7862ca2b46ac4d832263e5ac53eb0ed1a4550468e2e#4747200
this is the cursor: 0x06ced5394eda5d75dfdf702eb0259b8ad299383887e14e46a4e35ba55ff46a62#4837823
this is the cursor: 0x07e98339565c1c6d026e788bf41d9ea66b8f1e8003163abd9771ef9b9ca43b2e#4740715
this is the cursor: 0x090a99b2e4283a6e41f35279ff7c902108d50f91a1ea826c6ae93d9d012084d4#4633161
this is the cursor: 0x0a2c74b3ea49be2d9c9e6028ff0c0bbf365dd61d18d88bca863b3dfbed46486a#4677668
this is the cursor: 0x0b4b4d67db91930109060723bfac12cf7b539124728e905dcbc8fd9a731a2576#4840077
this is the cursor: 0x0c6dd5b75dcf24bc4b7f39bd5b9d

In [239]:
swapshhh = swaps
swapshhh.head()

,id,timestamp,sender,recipient,origin,amount0,amount1,amountUSD,sqrtPriceX96,tick,logIndex
0,0x00005be3feffa40600371196f6ebe99484cef2e030cb...,1679680919,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x697d1eeb266aec5251901e20e6a09dbdb68008c1,0x697d1eeb266aec5251901e20e6a09dbdb68008c1,-19922.526611,11.442382101497925129,19903.00432076901806488783284947735,1898284365247915042890667106569354,201692,295
1,0x00005e650a5a29a7340961ee1070bcb4d699a5c6b47c...,1680218951,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xc0c441bd9889f30fd219478a985e84fac8c2a706,1104.872406,-0.617085078176457911,1103.966933243716954948609702966833,1872856442624146468221690277382731,201422,369
2,0x00007ee86c8dedf910e254eb499746b9094d0d91ad3d...,1681392455,0x1111111254eeb25477b68fb85ed929f73a960582,0xbd4dbe0cb9136ffb4955ede88ebd5e92222ad09a,0x9d263af3950c80e3b49abd6def91f803762d65ab,-3951.498533,1.973163266992487917,3951.598149058666973024737265603912,1769998147893954485918035417674482,200293,75
3,0x00010099f534a61a99fa906e57cabd736e9f7e43ee0f...,1679774651,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xd22125cb225f39fe6aa0c16afe6b75872a3529d1,0xd22125cb225f39fe6aa0c16afe6b75872a3529d1,-12033.180287,7,12040.45320764383144992175201712345,1910435929075656551647679638334639,201820,272
4,0x00012828605998001bc178c99f7d95c467c7f481db6e...,1681089035,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x6992435f9a7bf34d5dedcad401e6ce8067cbb796,3299.547442,-1.774376095399703369,3302.745106341691833212028810929626,1837736537068722335967806380211673,201044,354


In [287]:
swapshhh["price"] = swapshhh["sqrtPriceX96"].astype(float)
swapshhh["sqrtPriceX96"] = swapshhh["price"].astype(float)

swapshhh["price"] = swapshhh["sqrtPriceX96"].apply(lambda x: utils.sqrtPrice2Price(x))
swapshhh["current_position0"] = utils.calculate_current_position0(l, swapshhh["price"])
swapshhh["current_position1"] = utils.calculate_current_position1(l, swapshhh["price"])

In [288]:
swapshhh["current_position0"][0] / (10 ** decimal0)

49298.90860733698

In [289]:
swapshhh["current_position1"][0] / (10 ** decimal1)

28.30092816216908

In [250]:
swapshhh["current_position1"][0]
28300928
100000000000
28300928162169.082

100000000000

In [317]:
# 计算fee
# convert str into float
swaps['amount0'] = swaps['amount0'].astype(float)
swaps['amount1'] = swaps['amount1'].astype(float)
swaps['amountUSD'] = swaps['amountUSD'].astype(float)
swaps['sqrtPriceX96'] = swaps['sqrtPriceX96'].astype(float)
swaps['tick'] = swaps['tick'].astype(int)
swaps['logIndex'] = swaps['logIndex'].astype(int)

In [318]:
# 现在来加总然后计算手续费
swaps1 = swaps
# 先把不用的列去掉
swaps1 = swaps1.drop(swaps1.columns[[0, 2, 3, 4, 7, 10]], axis = 1)
swaps1.head(6)

,timestamp,amount0,amount1,sqrtPriceX96,tick,price,current_position0,current_position1
192535,1679155211,-7168.364010,4.002126,1.871575e+33,201409,5.580273e+08,5.000245e+10,2.790273e+19
212529,1679155223,1792.878253,-1.000000,1.871598e+33,201409,5.580407e+08,5.000185e+10,2.790307e+19
196934,1679155223,-3027.044892,1.690060,1.871600e+33,201409,5.580420e+08,5.000179e+10,2.790310e+19
7410,1679155223,-17615.760520,9.835104,1.871596e+33,201409,5.580399e+08,5.000189e+10,2.790305e+19
152702,1679155235,1400.000000,-0.780866,1.871596e+33,201409,5.580397e+08,5.000189e+10,2.790304e+19
148420,1679155247,-347668.568355,194.152787,1.872009e+33,201413,5.582861e+08,4.999086e+10,2.790920e+19


In [319]:
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)

this is the cursor: 1682499600


In [320]:
liquidity_data

,periodStartUnix,liquidity,sqrtPrice,token0Price,token1Price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,tvlUSD,volumeToken0,volumeToken1,volumeUSD,feesUSD,txCount,open,high,low,close
0,1679158800,31691854789759819627,1871666948661873416199899917784088,1791.851542392230339967547914533666,0.0005580819483878331967604985081167192,201410,2314975469199928457298700227744982,1111162292302232016625192018264273618935405,362517939.1226720089381000671278629,0,0,0,0,372,1798.390894813620679474180985197866,1803.236399827628569085607995221095,1769.953587075568576406795835267567,1791.851542392230339967547914533666
1,1679162400,32915185914729732691,1867893400320714844515651086096830,1799.098710137099104209582500255936,0.0005558338707962253356429190522474716,201369,2315038215475779131181192686558395,1111189202831556094328153678234461295869361,363811618.6075028876192885997227013,0,0,0,0,366,1790.640320160079804025064362917098,1799.108571408040728063168297875355,1786.686354081499257894742581324593,1799.098710137099104209582500255936
2,1679166000,34369179026466248085,1869482743679436927327266292647007,1796.040997899866322473028892703993,0.0005567801632419932350116030494839081,201386,2315126854003039415933596971303595,1111241826769754789231620166343530092470985,363967492.8611761592527996744286012,0,0,0,0,333,1799.099225334904656531075427168004,1809.622901299084675474781496124223,1794.120699231514675114167039962852,1796.040997899866322473028892703993
3,1679169600,34183163223601833767,1868781276195693273308669229878293,1797.389578204559120182790388692096,0.000556362411424970994583196746389777,201379,2315168855482994101747218874079807,1111263525863761589158339866419647898430929,363205815.9317762712257394030005428,0,0,0,0,317,1796.054690522833853305120372567828,1805.57781837591635090288239389413,1792.725482994450340228488201862034,1797.389578204559120182790388692096
4,1679173200,34246952167731560476,1875752556320845172108591732547304,1784.054320332921814271944303424757,0.0005605210495011106585938907612211571,201453,2315193497548537858877306471738285,1111292420021385719852093400706179840724576,360909125.4200933323216227809336806,0,0,0,0,309,1797.199358810234823401563218695319,1797.220201287329445413990798860576,1783.038903387140753714413070004309,1784.054320332921814271944303424757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,1682485200,21099513284256475236,1832212390766229784081927199736230,1869.853302888126439066099105002248,0.000534801312196751579708054479268464,200984,2355721979099424391375588433074317,1132967572498394079568605140306372402023176,456946116.674010652148630961695913,0,0,0,0,154,1866.719859584542724539011713101046,1870.18957843412616640366826963631,1866.719859584542724539011713101046,1869.853302888126439066099105002248
925,1682488800,20082828377118460290,1834322954389487956409484348537620,1865.552888970253552918333422518791,0.0005360341193821522958527974158965844,201007,2355742416033158008569947010732748,1132982937267843117937972397755462415131951,456943740.5379049725979322965388548,0,0,0,0,186,1869.839920863508924503792223728627,1873.228427954734676867648340056136,1863.982120047554725303692790841834,1865.552888970253552918333422518791
926,1682492400,19995884481706635261,1836204576523764397010566592931715,1861.731455227726175798541147209244,0.0005371343956143666478455022890286458,201027,2355753034594320381486382973484870,1132992680528621822582070084384479920055761,457741471.66292047881249738559123,0,0,0,0,158,1865.554677454565063989986669062083,1866.984424260167882485730240407934,1859.95664916133377745250325787839,1861.731455227726175798541147209244
927,1682496000,104017415955513909378,1828807909176819812497820848593318,1876.821565030269585986929923758181,0.0005328157021596626258332084159883472,200946,2355803974092706558541561239644531,1133004015807783318210932404272416649904898,460373521.5231567167474889341272503,0,0,0,0,236,1861.73573099582713422279669304002,1879.057380260136590702286545504953,18

In [321]:
liquidity_data = liquidity_data[["periodStartUnix", "liquidity"]]
liquidity_data = liquidity_data.rename(columns={"periodStartUnix": "timestamp", "liquidity": "liquidity"})
# 先排序
swaps["timestamp"] = swaps["timestamp"].astype(int)
swaps = swaps.sort_values(by="timestamp")
liquidity_data = liquidity_data.sort_values(by="timestamp")
# 找liquidity data只时间最接近的来合并
merged_data = pd.merge_asof(
    swaps,
    liquidity_data[["timestamp", "liquidity"]],
    on="timestamp",
    direction="nearest"
)
merged_data.head(3)

,id,timestamp,sender,recipient,origin,amount0,amount1,amountUSD,sqrtPriceX96,tick,logIndex,price,current_position0,current_position1,liquidity
0,0xdb0dc52d01c31207494402aadc51386666b6bc7a5723...,1679155211,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x3d889b6391506032f9e9380a8f5cd3199661fb72,0x3d889b6391506032f9e9380a8f5cd3199661fb72,-7168.364010,4.002126,7167.505228,1.871575e+33,201409,498,5.580273e+08,5.000245e+10,2.790273e+19,31691854789759819627
1,0xf20cd95e06fae029ea9d9b496ec03b53488e905796ee...,1679155223,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xfb8c0777051218d3ecb6ade85783b64ce54ad409,1792.878253,-1.000000,1791.794058,1.871598e+33,201409,414,5.580407e+08,5.000185e+10,2.790307e+19,31691854789759819627
2,0xe020186967260f387a3e62e7bcd91697b104a2415048...,1679155223,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xa311f7ca3eb2fb98123a807f9b8e4bbbdbdcb2ee,0xa311f7ca3eb2fb98123a807f9b8e4bbbdbdcb2ee,-3027.044892,1.690060,3026.726007,1.871600e+33,201409,246,5.580420e+08,5.000179e+10,2.790310e+19,31691854789759819627


In [322]:
# 获取头寸的liquidity，后续手续费的分成是按照头寸的liquidity占总liquidity的比例来计算的
position_liquidity = l
merged_data['liquidity'] = merged_data['liquidity'].astype(float)
merged_data['ratio'] = float(position_liquidity) / merged_data["liquidity"]
merged_data['amount0'] = merged_data['amount0'].astype(float).apply(lambda x: max(0, x))
merged_data['amount1'] = merged_data['amount1'].astype(float).apply(lambda x: max(0, x))

merged_data["fee0"] = merged_data["amount0"] * merged_data['ratio'] * 0.0005
fee0_sum = merged_data["fee0"].sum()

merged_data["fee1"] = merged_data["amount1"] * merged_data['ratio'] * 0.0005
fee1_sum = merged_data["fee1"].sum()

print(fee0_sum)
print(fee1_sum)

149.70180069551822
0.08098251137828956


*** 

以下是用invest metric来计算投资指标的内容

In [343]:
from investMetric import metric
reload(metric)

<module 'investMetric.metric' from '/Users/bcfh/Documents/GitHub/learn_uniswapv3/strategies/../investMetric/metric.py'>

In [344]:
begin = datetime(2023, 3, 19)
end = datetime.now()
days = (end - begin).days
days

38

In [345]:
rf = 0.017
metric.metric(merged_data=merged_data, token0_amount=token0_amount, token1_amount=token1_amount, begin=begin, end=end, decimal0=decimal0, decimal1=decimal1, rf=rf, l=l)

** volatility =  1.338821970758587e+30
** roi =  0.01903507996529763
** sharpe_ratio =  0.01903507996529763
** max_drawdown =  0.008596415428852377
